
# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>

Le projet a été développé à l'aide de Alice Breton, étudiante à la maîtrise en génie informatique. Elle a suivi le cours lors de la session Hiver 2019.



# Projet : Débordement d'égouts

La description du projet est disponible à l'adresse suivante :
https://www.kaggle.com/t/a238b752c33a41d9803c2cdde6bfc929

Ce calepin Jupyter de base permet de charger et de nettoyer les données fournies. La dernière section détaille la génération du fichier des prédictions afin de le soumettre sur Kaggle dans le bon format.

Dans un premier temps, vous devrez récupérer l'archive *data.zip* sur Moodle. Ce dossier contient les fichiers suivants :
- surverses.csv
- precipitation.csv
- ouvrages-surverses.csv
- test.csv

Veuillez le décompresser dans le répertoire de ce calepin.

Le fichier *surverse.csv* répertorie s'il y a surverse (1) ou non (0) au cours de la journée pour les 170 ouvrages de débordement de 2013 à 2018 pour les mois de mai à octobre (inclusivement). Des renseignements additionnels sur les données sont disponibles à l'adresse suivante :

http://donnees.ville.montreal.qc.ca/dataset/debordement


Le fichier *precipitation.csv* contient les précipitations horaires en dixième de *mm* enregistrées à 5 stations pluviométriques de 2013 à 2019 :
- McTavish (7024745)
- Ste-Anne-de-Bellevue (702FHL8)
- Montreal/Pierre Elliott Trudeau Intl (702S006)
- Montreal/St-Hubert (7027329)
- L’Assomption (7014160)

Plus d'informations sur les précipitations sont disponibles à l'adresse suivante :

https://climat.meteo.gc.ca/climate_data/hourly_data_f.html?hlyRange=2008-01-08%7C2019-11-12&dlyRange=2002-12-23%7C2019-11-12&mlyRange=%7C&StationID=30165&Prov=QC&urlExtension=_f.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2019&selRowPerPage=25&Line=17&searchMethod=contains&Month=11&Day=12&txtStationName=montreal&timeframe=1&Year=2019

Le fichier *ouvrages-surverses.csv* contient différentes caractéristiques des ouvrages de débordement. 

http://donnees.ville.montreal.qc.ca/dataset/ouvrage-surverse

Le fichier *test.csv* contient les ouvrages et les jours pour lesquels vous devez prédire s'il y a eu surverse (true) ou non (false). Notez que l'on s'intéresse ici à 5 ouvrages de débordement localisés tout autour de l'Ile de Montréal :
- 3260-01D dans Rivière-des-Prairies 
- 3350-07D dans Ahunstic 
- 4240-01D dans Pointe-aux-Trembles 
- 4350-01D dans le Vieux-Montréal 
- 4380-01D dans Verdun

#### Remarque

Dans le projet, on ne s'intéresse qu'aux surverses occasionnées par les précipitations. On ignore les surverses occasionnées par 
- fonte de neige (F)
- travaux planifiés et entretien (TPL)
- urgence (U)
- autre (AUT)

On suppose que lorsqu'il n'y a pas de raison pour la surverse, il s'agit d'une surverse causée par les précipitations. Puisque Nous nous intéresserons uniquement aux surverses occasionnées par les précipitations liquides, nous ne considérons que les mois de mai à octobre inclusivement.

In [ ]:
using CSV, DataFrames, Statistics, Dates, Gadfly, LinearAlgebra, Random
using ScikitLearn, DecisionTree

@sk_import decomposition: PCA
@sk_import linear_model: LogisticRegression
@sk_import preprocessing: StandardScaler
@sk_import model_selection: (train_test_split, RandomizedSearchCV, GridSearchCV)
@sk_import metrics: (classification_report, roc_auc_score, auc, make_scorer, f1_score)
# @sk_import model_selection: train_test_split



#, GLM,Distributions,
include("functions.jl");

# Chargement des données et nettoyage préliminaire

## Chargement et nettoyage des surverses et précipitations

In [ ]:
data = CSV.read("data/surverses.csv",missingstring="-99999")
first(data,5)

# Extraction des surverses pour les mois de mai à octobre inclusivement
data = filter(row -> month(row.DATE) > 4, data) 
data = filter(row -> month(row.DATE) < 11, data) 

# Remplacement des valeurs *missing* dans la colonne :RAISON par "Inconnue"
raison = coalesce.(data[:,:RAISON],"Inconnue")
data[!,:RAISON] = raison

# Exlusion des surverses coccasionnées par d'autres facteurs que les précipitations liquides
data = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], data) 
select!(data, [:NO_OUVRAGE, :DATE, :SURVERSE])

surverse_df = dropmissing(data, disallowmissing=true)

### CHARGEMENT DES PRÉCIPITATIONS
data = CSV.read("data/precipitations.csv",missingstring="-99999")
rename!(data, Symbol("St-Hubert")=>:StHubert)

# Nettoyage des données sur les précipitations
data = filter(row -> month(row.date) > 4, data) 
data = filter(row -> month(row.date) < 11, data);


In [ ]:
# REMPLACER LES VALEURS MANQUANTES DANS LE DATAFRAME DE PRÉCIPITATIONS
# On veut remplacer les données manquante grâce à une simple régression linéaire (Ridge).
# Pour ce faire, chaque station météo agira comme variable explicative pour les 4 autres
# On drop les rangées dont les 5valeurs sont missing

data_full = deepcopy(data)
dropmissing!(data, disallowmissing=true)

models = Dict()

for i = 1:length(data_full[:, 1])
    if any(ismissing, data_full[i, 3:7]) #ismissing c est pas une variable mais une méthode de Julia, 3:7 pour les 5 stations
        missing = []
        notmissing = []
        for j=3:7 #on passe à travers les 5 colonnes pour regarder celles qui sont missing ou non
            if ismissing(data_full[i, j])
                push!(missing, names(data_full)[j]) #push généralisé à tous les conteneurs
                #Le point d exclamation pour dire à la fonction de modifier le conteneur qu on lui passe
            else
                push!(notmissing, names(data_full)[j])
            end
        end
        
        if length(notmissing) > 0 #si on a 5 missing sur la ligne on peut juste rien prédire donc on skip ça
            for m in missing #m c est la station à prédire
                if !haskey(models, (m, notmissing)) #ça véfirie si dans le dictionnaire models il y a la key (m, notmissing)
                    models[(m, notmissing)] = CoeffRidge(data, m, notmissing) #on change la case dans le dico palr le coef que la fonction nous retourne
                end
                data_full[i, m] = round(convert(Vector{Float64}, data_full[i, notmissing])' * models[(m, notmissing)]) # on ajoute à la case manque la nouvelle valeur = x*Beta
            end
        end
    end
end

dropmissing!(data_full);



## Enlever les données aberrantes somehow

In [ ]:
data_full[3902, 3] = 0
data_full[3902, 7] = 0

## Extraction variables explicatives (Maude L.)

In [ ]:
# # Maximum de précipitations en 1 heure dans une journée.
# X_pcp_max = by(data_full, :date,  MAXMcTavish = :McTavish=>maximum, MAXBellevue = :Bellevue=>maximum, 
#    MAXAssomption = :Assomption=>maximum, MAXTrudeau = :Trudeau=>maximum, MAXStHubert = :StHubert=>maximum)

# # Somme des précipitations dans une journée
# X_pcp_sum = by(data_full, :date,  SUMMcTavish = :McTavish=>sum, SUMBellevue = :Bellevue=>sum, 
#    SUMAssomption = :Assomption=>sum, SUMTrudeau = :Trudeau=>sum, SUMStHubert = :StHubert=>sum)

# # Précipitation à chaque heure pour chaque journée
# X_pcp_hour = data_full

# # Somme des précipitations des 2 heures précédentes

# X_pcp_three_hours = copy(data_full)
# buffer = copy(data_full)
# for i=3:7
#     for j=1:length(data_full[:,2])
#         if (j > 2)
#             X_pcp_three_hours[j, i] = (buffer[(j-2), i] + buffer[(j-1), i] + buffer[j,i])
#         else
#             X_pcp_three_hours[j, i] = buffer[j, i]
#         end
#     end
# end

# # Somme des précipitations des 2 jours précédents

# X_pcp_two_days = DataFrame(date= X_pcp_sum[:,:date], SUM2DaysMcTavish=zeros(Int64, length(X_pcp_sum[:,1])), SUM2DaysBellevue=zeros(Int64, length(X_pcp_sum[:,1])), SUM2DaysAssomption=zeros(Int64, length(X_pcp_sum[:,1])), SUM2DaysTrudeau=zeros(Int64, length(X_pcp_sum[:,1])), SUM2DaysStHubert=zeros(Int64, length(X_pcp_sum[:,1])))
# #buffer = copy(X_pcp_sum)
# for i=2:6
#     for j=1:length(X_pcp_two_days[:,1])
#         if (j > 1)
#             X_pcp_two_days[j, i] = X_pcp_sum[(j-1), i] + X_pcp_sum[j,i]
#         else
#             X_pcp_two_days[j, i] = X_pcp_sum[j, i]
#         end
#     end
# end


# # Mois de la prédiction pour chaque ouvrage
# X_date = DataFrame(NO_OUVRAGE=surverse_df[:,:NO_OUVRAGE], MOIS=zeros(Int64,length(surverse_df[:,1])), SURVERSE=surverse_df[:,:SURVERSE])
# for i=1:length(surverse_df[:,1])
#     X_date[i,:MOIS] = month(surverse_df[i,:DATE])
# end

# X_mois = DataFrame(MOIS=["Mai", "Juin", "Juillet", "Août", "Septembre", "Octobre"], RivierePrairie=zeros(Int64,6), Ahuntsic=zeros(Int64,6), PointeAuxTrembles=zeros(Int64,6), VieuxMontreal=zeros(Int64,6), Verdun=zeros(Int64,6))
# for j=1:length(X_date[:,1])
#     if(X_date[j,1] == "3260-01D")
#         X_mois[(X_date[j,2]-4),:RivierePrairie] += (X_date[j,3])
#     elseif (X_date[j,1] == "3350-07D")
#         X_mois[(X_date[j,2]-4),:Ahuntsic] += (X_date[j,3])
#     elseif (X_date[j,1] == "4240-01D")
#         X_mois[(X_date[j,2]-4),:PointeAuxTrembles] += (X_date[j,3])
#     elseif (X_date[j,1] == "4350-01D")
#         X_mois[(X_date[j,2]-4),:VieuxMontreal] += (X_date[j,3])
#     elseif (X_date[j,1] == "4380-01D")
#         X_mois[(X_date[j,2]-4),:Verdun] += (X_date[j,3])
#     end
# end

##Anis
# Maximum de précipitations en 1 heure dans une journée.
X_pcp_max = by(data_full, :date,  MAXMcTavish = :McTavish=>maximum, MAXBellevue = :Bellevue=>maximum, 
   MAXAssomption = :Assomption=>maximum, MAXTrudeau = :Trudeau=>maximum, MAXStHubert = :StHubert=>maximum)

# Somme des précipitations dans une journée
X_pcp_sum = by(data_full, :date,  SUMMcTavish = :McTavish=>sum, SUMBellevue = :Bellevue=>sum, 
   SUMAssomption = :Assomption=>sum, SUMTrudeau = :Trudeau=>sum, SUMStHubert = :StHubert=>sum)

# Précipitation à chaque heure pour chaque journée
X_pcp_hour = data_full

# Somme des précipitations des 2 heures précédentes

X_pcp_three_hours = copy(data_full)
buffer = copy(data_full)
for i=3:7
    for j=1:length(data_full[:,2])
        if (j > 2)
            X_pcp_three_hours[j, i] = (buffer[(j-2), i] + buffer[(j-1), i] + buffer[j,i])
        else
            X_pcp_three_hours[j, i] = buffer[j, i]
        end
    end
end

# Somme des précipitations des 2 jours précédents

X_pcp_two_days = DataFrame(date= X_pcp_sum[:,:date], SUM2DaysMcTavish=zeros(Int64, length(X_pcp_sum[:,1])), SUM2DaysBellevue=zeros(Int64, length(X_pcp_sum[:,1])), SUM2DaysAssomption=zeros(Int64, length(X_pcp_sum[:,1])), SUM2DaysTrudeau=zeros(Int64, length(X_pcp_sum[:,1])), SUM2DaysStHubert=zeros(Int64, length(X_pcp_sum[:,1])))
#buffer = copy(X_pcp_sum)
for i=2:6
    for j=1:length(X_pcp_two_days[:,1])
        if (j > 1)
            X_pcp_two_days[j, i] = X_pcp_sum[(j-1), i] + X_pcp_sum[j,i]
        else
            X_pcp_two_days[j, i] = X_pcp_sum[j, i]
        end
    end
end

# Moyenne des précipitations avec les 3 dernières heures

X_mean_pcp_three_hours = copy(data_full)
buffer = copy(data_full)
for i=3:7
    for j=1:length(data_full[:,3])
        if (j > 3)
            X_mean_pcp_three_hours[j, i] = floor((buffer[(j-3), i] + buffer[(j-2), i] + buffer[(j-1), i] + buffer[j,i]) / 4)
        else
            X_mean_pcp_three_hours[j, i] = buffer[j, i]
        end
    end
end

println(X_mean_pcp_three_hours[1:5, :])

# Mediane des précipitations en 1 heure dans une journée.
X_pcp_median = by(data_full, :date,  MEDIANMcTavish = :McTavish=>median, MEDIANBellevue = :Bellevue=>median, 
   MEDIANAssomption = :Assomption=>median, MEDIANTrudeau = :Trudeau=>median, MEDIANStHubert = :StHubert=>median)


# Mois de la prédiction pour chaque ouvrage
X_date = DataFrame(NO_OUVRAGE=surverse_df[:,:NO_OUVRAGE], MOIS=zeros(Int64,length(surverse_df[:,1])), SURVERSE=surverse_df[:,:SURVERSE])
for i=1:length(surverse_df[:,1])
    X_date[i,:MOIS] = month(surverse_df[i,:DATE])
end

X_mois = DataFrame(MOIS=["Mai", "Juin", "Juillet", "Août", "Septembre", "Octobre"], RivierePrairie=zeros(Int64,6), Ahuntsic=zeros(Int64,6), PointeAuxTrembles=zeros(Int64,6), VieuxMontreal=zeros(Int64,6), Verdun=zeros(Int64,6))
for j=1:length(X_date[:,1])
    if(X_date[j,1] == "3260-01D")
        X_mois[(X_date[j,2]-4),:RivierePrairie] += (X_date[j,3])
    elseif (X_date[j,1] == "3350-07D")
        X_mois[(X_date[j,2]-4),:Ahuntsic] += (X_date[j,3])
    elseif (X_date[j,1] == "4240-01D")
        X_mois[(X_date[j,2]-4),:PointeAuxTrembles] += (X_date[j,3])
    elseif (X_date[j,1] == "4350-01D")
        X_mois[(X_date[j,2]-4),:VieuxMontreal] += (X_date[j,3])
    elseif (X_date[j,1] == "4380-01D")
        X_mois[(X_date[j,2]-4),:Verdun] += (X_date[j,3])
    end
end


In [ ]:
# Création de 5 dataframes séparés pour les données de surverses des 5 ouvrages d'intérêt
ouvrage = ["3260-01D", "3350-07D", "4240-01D", "4350-01D", "4380-01D"]

filtered_surverse = filter(row -> row.NO_OUVRAGE == ouvrage[1], surverse_df)
surverse_rivierePrairie = DataFrame(DATE=filtered_surverse[:, 2], SURVERSE=filtered_surverse[:, 3])

filtered_surverse = filter(row -> row.NO_OUVRAGE == ouvrage[2], surverse_df)
surverse_ahuntsic = DataFrame(DATE=filtered_surverse[:, 2], SURVERSE=filtered_surverse[:, 3])

filtered_surverse = filter(row -> row.NO_OUVRAGE == ouvrage[3], surverse_df)
surverse_pointeAuxTrembles = DataFrame(DATE=filtered_surverse[:, 2], SURVERSE=filtered_surverse[:, 3])

filtered_surverse = filter(row -> row.NO_OUVRAGE == ouvrage[4], surverse_df)
surverse_vieuxMontreal = DataFrame(DATE=filtered_surverse[:, 2], SURVERSE=filtered_surverse[:, 3])

filtered_surverse = filter(row -> row.NO_OUVRAGE == ouvrage[5], surverse_df)
surverse_verdun = DataFrame(DATE=filtered_surverse[:, 2], SURVERSE=filtered_surverse[:, 3]);


## Création de variables explicatives représentant la densité de pluie tombée selon les heures
Afin de prendre en considération une quantité de précipitations importantes tombées en une courte période de temps, nous avons calculé pour chaque heure de la journée, la somme des précipitations des deux heures précédentes avec les précipitations de l'heure courante. À partir de cela, nous avons créé la variable MAXPCP3HOURS qui présente la quantité maximale de pluie par bloc de 3 heures dans une journée, et la variable NUMBLOCK qui représente le nombre de bloc de 3 heures ayant une somme de précipitations plus élevé qu'un seuil déterminé en essayant différents seuil et en représentant graphiquement la discrimination entre les journées avec surverse et sans surverse, selon le seuil choisi.

In [ ]:
function aboveThreshold(x::Array{Int64,1}, threshold::Int64)
    num = 0
    for j=1:length(x)
        if x[j] > threshold
            num += 1
        end
    end
    return num
end

In [ ]:
## Dataframe avec le maximum de précipitations dans chaque bloc de 3 heures, et le nombre de bloc de 3 heures
## ayant eu plus que 75 mm de précipitations.
df_3h = DataFrame(date=X_pcp_sum[:, :date], MAXBLOCMcTavish=zeros(Int64, length(X_pcp_sum[:,1])), MAXBLOCBellevue=zeros(Int64, length(X_pcp_sum[:,1])), MAXBLOCAssomption=zeros(Int64, length(X_pcp_sum[:,1])), MAXBLOCTrudeau=zeros(Int64, length(X_pcp_sum[:,1])), MAXBLOCStHubert=zeros(Int64, length(X_pcp_sum[:,1])), BLOCMcTavish=zeros(Int64, length(X_pcp_sum[:,1])), BLOCBellevue=zeros(Int64, length(X_pcp_sum[:,1])), BLOCAssomption=zeros(Int64, length(X_pcp_sum[:,1])), BLOCTrudeau=zeros(Int64, length(X_pcp_sum[:,1])), BLOCStHubert=zeros(Int64, length(X_pcp_sum[:,1])))
for i=1:length(X_pcp_sum[:,1])-1
    for j=2:6
        df_3h[i, j] = findmax(X_pcp_three_hours[((i-1)*24)+1:i*24, j+1])[1]
    end
    for j=7:11
        df_3h[i, j] = aboveThreshold(X_pcp_three_hours[((i-1)*24)+1:i*24, j-4], 75)
    end
end

In [ ]:
## X_max_3h est la variable avec le maximum de précipitations par 3 heures
## X_bloc_3h est la variable avec le nombre de blocs ayant eu plus que 75 mm de précipitations
X_max_3h = df_3h[!, 1:6]
X_bloc_3h = df_3h[!, 7:11]
insert!(X_bloc_3h, 1, df_3h[!, 1], :date);

In [ ]:
## Dataframe pour le plot d'en dessous
df_3h_3260 = copy(df_3h)
df_3h_3260.SURVERSE = zeros(Int64, length(df_3h[:,1]))
missing = []
for i=1:length(df_3h_3260[:,1])
    surverse = filter(x -> x.DATE == df_3h[i, :date], surverse_vieuxMontreal).SURVERSE
    if length(surverse) != 0
        df_3h_3260[i, :SURVERSE] = surverse[1]
    else
        push!(missing, i)
    end
end
deleterows!(df_3h_3260, missing);

In [ ]:
## Plot pour déterminer ish le seuil de précipitations par bloc de 3 heures
plot(df_3h_3260,x="MAXBLOCMcTavish", y="BLOCMcTavish", color=:SURVERSE, Geom.beeswarm)

# Analyse exploratoire

Cette section consitue une analyse exploratoire superficielle permettant de voir s'il existe un lien entre les précipitations et les surverses.

Prenons arbitrairement l'ouvrage de débordement près du Bota-Bota (4350-01D). La station météorologique la plus proche est McTavish. Prenons deux variables explicatives simple :
- la somme journalière des précipitations
- le taux horaire maximum journalier de précipitations

#### Calcul de la quantité journalière de précipitations pour chacune des stations météorologiques

# Modèle #1: Classification par Random Forests

### Écrire une description 

In [14]:
# # Construction d'un dataframe contenant toutes les variables explicatives

# function CreateDataframe(ouvrage, surverse_df)
#     df_train = filter(row -> row.NO_OUVRAGE == ouvrage, surverse_df)

#     x₁ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière McTavish
#     x₂ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière Bellevue
#     x₃ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière Assomption
#     x₄ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière PET
#     x₅ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière St-Hubert
#     x₆ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours McTavish
#     x₇ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours Bellevue
#     x₈ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours Assomption
#     x₉ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours PET
#     x₁₀ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours St-Hubert
#     x₁₁ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier McTavish
#     x₁₂ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier Bellevue
#     x₁₃ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier Assomption
#     x₁₄ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier PET
#     x₁₅ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier St-Hubert
#     x₁₆ = Array{Int64}(undef, size(df_train,1)) # variable pour le max par bloc de 3h McTavish
#     x₁₇ = Array{Int64}(undef, size(df_train,1)) # variable pour le max par bloc de 3h Bellevue
#     x₁₈ = Array{Int64}(undef, size(df_train,1)) # variable pour le max par bloc de 3h Assomption
#     x₁₉ = Array{Int64}(undef, size(df_train,1)) # variable pour le max par bloc de 3h PET
#     x₂₀ = Array{Int64}(undef, size(df_train,1)) # variable pour le max par bloc de 3h St-Hubert
#     x₂₁ = Array{Int64}(undef, size(df_train,1)) # variable pour le nombre de blocs de 3h avec plus de 75 mm de pluie McTavish
#     x₂₂ = Array{Int64}(undef, size(df_train,1)) # variable pour le nombre de blocs de 3h avec plus de 75 mm de pluie Bellevue
#     x₂₃ = Array{Int64}(undef, size(df_train,1)) # variable pour le nombre de blocs de 3h avec plus de 75 mm de pluie Assomption
#     x₂₄ = Array{Int64}(undef, size(df_train,1)) # variable pour le nombre de blocs de 3h avec plus de 75 mm de pluie PET
#     x₂₅ = Array{Int64}(undef, size(df_train,1)) # variable pour le nombre de blocs de 3h avec plus de 75 mm de pluie St-Hubert



#     for i=1:size(df_train,1)

#         ind = findfirst(X_pcp_sum[:,:date] .== df_train[i,:DATE])
        
#         x₁[i] = X_pcp_sum[ind,:SUMMcTavish]

#         x₂[i] = X_pcp_sum[ind,:SUMBellevue]

#         x₃[i] = X_pcp_sum[ind,:SUMAssomption]

#         x₄[i] = X_pcp_sum[ind,:SUMTrudeau]

#         x₅[i] = X_pcp_sum[ind,:SUMStHubert]
        
#         ind = findfirst(X_pcp_two_days[:,:date] .== df_train[i,:DATE])

#         x₆[i] = X_pcp_two_days[ind, :SUM2DaysMcTavish]

#         x₇[i] = X_pcp_two_days[ind, :SUM2DaysBellevue]

#         x₈[i] = X_pcp_two_days[ind, :SUM2DaysAssomption]

#         x₉[i] = X_pcp_two_days[ind, :SUM2DaysTrudeau]

#         x₁₀[i] = X_pcp_two_days[ind, :SUM2DaysStHubert]
        
#         ind = findfirst(X_pcp_max[:,:date] .== df_train[i,:DATE])

#         x₁₁[i] = X_pcp_max[ind, :MAXMcTavish]

#         x₁₂[i] = X_pcp_max[ind, :MAXBellevue]

#         x₁₃[i] = X_pcp_max[ind, :MAXAssomption]

#         x₁₄[i] = X_pcp_max[ind, :MAXTrudeau]

#         x₁₅[i] = X_pcp_max[ind, :MAXStHubert]
        
#         ind = findfirst(X_max_3h[:,:date] .== df_train[i,:DATE])
        
#         x₁₆[i] = X_max_3h[ind, :MAXBLOCMcTavish]
        
#         x₁₇[i] = X_max_3h[ind, :MAXBLOCBellevue]
        
#         x₁₈[i] = X_max_3h[ind, :MAXBLOCAssomption]
        
#         x₁₉[i] = X_max_3h[ind, :MAXBLOCTrudeau]
        
#         x₂₀[i] = X_max_3h[ind, :MAXBLOCStHubert]
        
#         ind = findfirst(X_bloc_3h[:,:date] .== df_train[i,:DATE])
        
#         x₂₁[i] = X_bloc_3h[ind, :BLOCMcTavish]
        
#         x₂₂[i] = X_bloc_3h[ind, :BLOCBellevue]
        
#         x₂₃[i] = X_bloc_3h[ind, :BLOCAssomption]
        
#         x₂₄[i] = X_bloc_3h[ind, :BLOCTrudeau]
        
#         x₂₅[i] = X_bloc_3h[ind, :BLOCStHubert]

#     end

#     df_train[!,:SUMMcTavish] = x₁
#     df_train[!,:SUMBellevue] = x₂
#     df_train[!,:SUMAssomption] = x₃
#     df_train[!,:SUMStHubert] = x₄
#     df_train[!,:SUMPET] = x₅
#     df_train[!,:SUM2McTavish] = x₆
#     df_train[!,:SUM2Bellevue] = x₇
#     df_train[!,:SUM2Assomption] = x₈
#     df_train[!,:SUM2StHubert] = x₉
#     df_train[!,:SUM2PET] = x₁₀
#     df_train[!,:MAXMcTavish] = x₁₁
#     df_train[!,:MAXBellevue] = x₁₂
#     df_train[!,:MAXAssomption] = x₁₃
#     df_train[!,:MAXStHubert] = x₁₄
#     df_train[!,:MAXPET] = x₁₅
#     df_train[!, :MAXBLOCMcTavish] = x₁₆
#     df_train[!, :MAXBLOCBellevue] = x₁₇
#     df_train[!, :MAXBLOCAssomption] = x₁₈
#     df_train[!, :MAXBLOCTrudeau] = x₁₉
#     df_train[!, :MAXBLOCStHubert] = x₂₀
#     df_train[!, :BLOCMcTavish] = x₂₁
#     df_train[!, :BLOCBellevue] = x₂₂
#     df_train[!, :BLOCAssomption] = x₂₃
#     df_train[!, :BLOCTrudeau] = x₂₄
#     df_train[!, :BLOCStHubert] = x₂₅
    
#     return df_train

# end


# Construction d'un dataframe contenant toutes les variables explicatives

function CreateDataframe(ouvrage, surverse_df)
    df_train = filter(row -> row.NO_OUVRAGE == ouvrage, surverse_df)

    x₁ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière McTavish
    x₂ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière Bellevue
    x₃ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière Assomption
    x₄ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière PET
    x₅ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière St-Hubert
    x₆ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours McTavish
    x₇ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours Bellevue
    x₈ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours Assomption
    x₉ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours PET
    x₁₀ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours St-Hubert
    x₁₁ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier McTavish
    x₁₂ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier Bellevue
    x₁₃ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier Assomption
    x₁₄ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier PET
    x₁₅ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier St-Hubert
    x₁₆ = Array{Int64}(undef, size(df_train,1)) # variable pour le max par bloc de 3h McTavish
    x₁₇ = Array{Int64}(undef, size(df_train,1)) # variable pour le max par bloc de 3h Bellevue
    x₁₈ = Array{Int64}(undef, size(df_train,1)) # variable pour le max par bloc de 3h Assomption
    x₁₉ = Array{Int64}(undef, size(df_train,1)) # variable pour le max par bloc de 3h PET
    x₂₀ = Array{Int64}(undef, size(df_train,1)) # variable pour le max par bloc de 3h St-Hubert
    x₂₁ = Array{Int64}(undef, size(df_train,1)) # variable pour le nombre de blocs de 3h avec plus de 75 mm de pluie McTavish
    x₂₂ = Array{Int64}(undef, size(df_train,1)) # variable pour le nombre de blocs de 3h avec plus de 75 mm de pluie Bellevue
    x₂₃ = Array{Int64}(undef, size(df_train,1)) # variable pour le nombre de blocs de 3h avec plus de 75 mm de pluie Assomption
    x₂₄ = Array{Int64}(undef, size(df_train,1)) # variable pour le nombre de blocs de 3h avec plus de 75 mm de pluie PET
    x₂₅ = Array{Int64}(undef, size(df_train,1)) # variable pour le nombre de blocs de 3h avec plus de 75 mm de pluie St-Hubert
    x₂₆ = Array{Int64}(undef, size(df_train,1))    
    x₂₇ = Array{Int64}(undef, size(df_train,1))
    x₂₈ = Array{Int64}(undef, size(df_train,1))
    x₂₉ = Array{Int64}(undef, size(df_train,1))
    x₃₀ = Array{Int64}(undef, size(df_train,1))
    x₃₁ = Array{Int64}(undef, size(df_train,1))
    x₃₂ = Array{Int64}(undef, size(df_train,1))
    x₃₃ = Array{Int64}(undef, size(df_train,1))
    x₃₄ = Array{Int64}(undef, size(df_train,1))
    x₃₅ = Array{Int64}(undef, size(df_train,1))



    for i=1:size(df_train,1)

        ind = findfirst(X_pcp_sum[:,:date] .== df_train[i,:DATE])
        
        x₁[i] = X_pcp_sum[ind,:SUMMcTavish]

        x₂[i] = X_pcp_sum[ind,:SUMBellevue]

        x₃[i] = X_pcp_sum[ind,:SUMAssomption]

        x₄[i] = X_pcp_sum[ind,:SUMTrudeau]

        x₅[i] = X_pcp_sum[ind,:SUMStHubert]
        
        ind = findfirst(X_pcp_two_days[:,:date] .== df_train[i,:DATE])

        x₆[i] = X_pcp_two_days[ind, :SUM2DaysMcTavish]

        x₇[i] = X_pcp_two_days[ind, :SUM2DaysBellevue]

        x₈[i] = X_pcp_two_days[ind, :SUM2DaysAssomption]

        x₉[i] = X_pcp_two_days[ind, :SUM2DaysTrudeau]

        x₁₀[i] = X_pcp_two_days[ind, :SUM2DaysStHubert]
        
        ind = findfirst(X_pcp_max[:,:date] .== df_train[i,:DATE])

        x₁₁[i] = X_pcp_max[ind, :MAXMcTavish]

        x₁₂[i] = X_pcp_max[ind, :MAXBellevue]

        x₁₃[i] = X_pcp_max[ind, :MAXAssomption]

        x₁₄[i] = X_pcp_max[ind, :MAXTrudeau]

        x₁₅[i] = X_pcp_max[ind, :MAXStHubert]
        
        ind = findfirst(X_max_3h[:,:date] .== df_train[i,:DATE])
        
        x₁₆[i] = X_max_3h[ind, :MAXBLOCMcTavish]
        
        x₁₇[i] = X_max_3h[ind, :MAXBLOCBellevue]
        
        x₁₈[i] = X_max_3h[ind, :MAXBLOCAssomption]
        
        x₁₉[i] = X_max_3h[ind, :MAXBLOCTrudeau]
        
        x₂₀[i] = X_max_3h[ind, :MAXBLOCStHubert]
        
        ind = findfirst(X_bloc_3h[:,:date] .== df_train[i,:DATE])
        
        x₂₁[i] = X_bloc_3h[ind, :BLOCMcTavish]
        
        x₂₂[i] = X_bloc_3h[ind, :BLOCBellevue]
        
        x₂₃[i] = X_bloc_3h[ind, :BLOCAssomption]
        
        x₂₄[i] = X_bloc_3h[ind, :BLOCTrudeau]
        
        x₂₅[i] = X_bloc_3h[ind, :BLOCStHubert]
        
        ind = findfirst(X_mean_pcp_three_hours[:,:date] .== df_train[i,:DATE])
        
        x₂₆[i] = X_mean_pcp_three_hours[ind, :McTavish]
        
        x₂₇[i] = X_mean_pcp_three_hours[ind, :Bellevue]
        
        x₂₈[i] = X_mean_pcp_three_hours[ind, :Assomption]
        
        x₂₉[i] = X_mean_pcp_three_hours[ind, :Trudeau]
        
        x₃₀[i] = X_mean_pcp_three_hours[ind, :StHubert]
        
        ind = findfirst(X_pcp_median[:,:date] .== df_train[i,:DATE])
        
        x₃₁[i] = floor(X_pcp_median[ind, :MEDIANMcTavish])
        
        x₃₂[i] = floor(X_pcp_median[ind, :MEDIANBellevue])
        
        x₃₃[i] = floor(X_pcp_median[ind, :MEDIANAssomption])
            
        x₃₄[i] = floor(X_pcp_median[ind, :MEDIANTrudeau])
        
        x₃₅[i] = floor(X_pcp_median[ind, :MEDIANStHubert])
        

    end

    df_train[!,:SUMMcTavish] = x₁
    df_train[!,:SUMBellevue] = x₂
    df_train[!,:SUMAssomption] = x₃
    df_train[!,:SUMStHubert] = x₄
    df_train[!,:SUMPET] = x₅
    df_train[!,:SUM2McTavish] = x₆
    df_train[!,:SUM2Bellevue] = x₇
    df_train[!,:SUM2Assomption] = x₈
    df_train[!,:SUM2StHubert] = x₉
    df_train[!,:SUM2PET] = x₁₀
    df_train[!,:MAXMcTavish] = x₁₁
    df_train[!,:MAXBellevue] = x₁₂
    df_train[!,:MAXAssomption] = x₁₃
    df_train[!,:MAXStHubert] = x₁₄
    df_train[!,:MAXPET] = x₁₅
    df_train[!, :MAXBLOCMcTavish] = x₁₆
    df_train[!, :MAXBLOCBellevue] = x₁₇
    df_train[!, :MAXBLOCAssomption] = x₁₈
    df_train[!, :MAXBLOCTrudeau] = x₁₉
    df_train[!, :MAXBLOCStHubert] = x₂₀
    df_train[!, :BLOCMcTavish] = x₂₁
    df_train[!, :BLOCBellevue] = x₂₂
    df_train[!, :BLOCAssomption] = x₂₃
    df_train[!, :BLOCTrudeau] = x₂₄
    df_train[!, :BLOCStHubert] = x₂₅
    df_train[!, :MEANMcTavish] = x₂₆
    df_train[!, :MEANBellevue] = x₂₇
    df_train[!, :MEANAssomption] = x₂₈
    df_train[!, :MEANTrudeau] = x₂₉
    df_train[!, :MEANStHubert] = x₃₀
    df_train[!, :MEDIANMcTavish] = x₃₁
    df_train[!, :MEDIANBellevue] = x₃₂
    df_train[!, :MEDIANAssomption] = x₃₃
    df_train[!, :MEDIANTrudeau] = x₃₄
    df_train[!, :MEDIANStHubert] = x₃₅
    
    
    return df_train

end


CreateDataframe (generic function with 1 method)

In [15]:
"""
getForest permet d'obtenir pour chaque ouvrage un modèle de prédiction basé sur la classification par forêts aléatoires. 
Le paramètre rf_fit_params correspondent aux valeurs possibles des hyperparamètres de RandomForestClassifier

@params
num_ouvrage:      le numéro de l'ouvrage à évaluer
rf_fit_param:     dictionnaire des valeurs possibles pour les hyperparamètres du modèle RF
n_folds:          le nombre de plis à effectuer sur l'ensemble d'entrainement lors de la validation croisée
grid:             false par défaut, si à true, la fonction effectue une GridSearch (exhaustive) sur l'ensemble des
                  hyperparamètres fournis, sinon elle effectue une RandomizedSearch
n_iter:           10 par défaut, le nombre de modèles à essayer lors d'une RandomizedSearch

"""
function getForest(num_ouvrage::String; fit_params::Dict, n_folds = 5, grid=false, n_iter = 1000, verbose=1)
    train = CreateDataframe(num_ouvrage, surverse_df)
    all_X = convert(Matrix{Float64}, train[:, 4:28])
    all_y = train[!, :SURVERSE]
    
    # On doit utiliser la fonction train_test_split de sklearn car notre jeu de données est très désiquilibré.
    # le parametre "stratify" de la fonction permet de parer à ca
    
    X_train, X_test, y_train, y_test = train_test_split(all_X, all_y, test_size=.2, shuffle=true, random_state = 34, stratify=all_y)

    # Préparation de la classification par RandomForests.
    # Les HyperParamètres seront évalués dans la GridSearch/RandomizedSearch
    # StandardScaler() est utilisé pour normaliser les données
    pipe = Pipelines.Pipeline([("ss", StandardScaler()),
                               ("rf", RandomForestClassifier(rng=42))])
    
    n_models = 1
    
    for (k,v) in fit_params
        n_models *= length(v)
    end

    println("Nombre de modèles à parcourir: $(n_models)\n")

    if grid
        M = GridSearch.GridSearchCV(estimator=pipe,
                                    param_grid=fit_params,
                                    cv=n_folds,
                                    verbose=verbose)
    else
        M = GridSearch.RandomizedSearchCV(estimator=pipe,
                                          param_distributions=fit_params,
                                          cv=n_folds,
                                          n_iter=n_iter,
                                          verbose=verbose)
    end

    bon_fit = fit!(M, X_train, y_train)
    println("training score: ", M.best_score_ , '\n', "best parameters: ", M.best_params_,'\n')
    y_true, y_pred = y_test, predict(M, X_test) 
    println("F1: ", f1_score(y_true, y_pred))
    println("AUC: ",  roc_auc_score(y_true, y_pred))
    println("test score: ", classification_report(y_true, y_pred))

    return bon_fit
end

getForest

# Random Forest et RandomizedSearch/GridSearch
## Démarche
#### 1. RandomizedSearch
Nous utilisons la technique de classification par RandomForests. Afin d'optimiser notre modèle, nous devons parcourir l'espace des hyperparamètres du modèle pour n'en garder que les meilleures valeurs: 
1. n_subfeatures: le nombre de variables explicatives à sélectionner aléatoirement pour chacun des arbres composant la forêt
2. n_trees: le nombre d'arbres composant la forêt
3. partial_sampling: la fraction du dataframe d'entrainement sur laquelle entrainer chacun des arbres
4. max_depth: la profondeur maximale de chacun des arbres
5. min_samples_leaf: le nombre minimal de données d'entrainement ("sample") par feuille
6. min_sample_split: le nombre minimal de données d'entrainement requis pour qu'un noeud de l'arbre se sépare en deux autres noeuds

Nous avons tout d'abord exécuté une RandomizedSearch de manière indépendante sur chacun des hyperparamètres en donnant à chaque fois une très grande portée à chaque paramètre. En modifiant ainsi chaque hyperparamètre un à un, nous avons pu réduire progressivement la portée possible de chaque paramètre en observant le score de chaque forêt en fonction du paramètre modifié. 

L'objectif de cette démarche a été de progressivement réduire l'espace des modèles pour pouvoir effectuer une GridSearch à la place d'une RandomizedSearch, et ce, sur l'entièreté des hyperparamètres

#### 2. GridSearch

La GridSearch, contrairement à la RandomizedSearch, permet de passer à travers chacun des modèles proposés, un à un. Comme pour la RandomizedSearch, elle utilise la validation croisée qui aide à réduire le phénomène de surapprentissage, en testant chacun des modèles plusieurs fois (k fois) sur un ensemble de test et d'entrainement différent à chaque fois.

In [ ]:
ouvrage = ["3260-01D", "3350-07D", "4240-01D", "4350-01D", "4380-01D"]
best_models_rf = Dict()

#Préparation d'un dictionnaire de paramètres pour l'optimisation des hyperparamètres du modèle
#On déclare les valeurs jugées possibles pour le modèle

rf_fit_params = Dict()
rf_fit_params["rf__n_subfeatures"] = 3:1:10 
rf_fit_params["rf__n_trees"] = 80:20:500
rf_fit_params["rf__partial_sampling"] = [0.6, 0.7, 0.8]
rf_fit_params["rf__max_depth"] = 3:2:50
rf_fit_params["rf__min_samples_leaf"] = 5:2:10
rf_fit_params["rf__min_samples_split"] = 2:2:16
rf_fit_params["rf__min_purity_increase"] = [0.0, 0.1, 0.2]

for i=1:length(ouvrage)
    @time best_models_rf[ouvrage[i]] = getForest(ouvrage[i],
                                              fit_params = rf_fit_params,
                                              n_iter=250)
    println("Accuracy score: ", best_models_rf[ouvrage[i]].best_score_, "\n")
#     println("Model score: ", best_models_rf[ouvrage[i]].best_estimator_, "\n")

    println(ouvrage[i], " done")
    println(" ******************************* ")
    println(" ******************************* ")

end


Nombre de modèles à parcourir: 912384

Fitting 5 folds for each of 250 candidates, totalling 1250 fits
training score: 0.9555302166476625
best parameters: Dict{Symbol,Any}(:rf__min_samples_leaf => 9,:rf__n_trees => 220,:rf__min_samples_split => 6,:rf__min_purity_increase => 0.1,:rf__n_subfeatures => 3,:rf__max_depth => 27,:rf__partial_sampling => 0.6)

F1: 0.7333333333333334
AUC: 0.7894736842105263
test score:               precision    recall  f1-score   support

           0       0.96      1.00      0.98       201
           1       1.00      0.58      0.73        19

    accuracy                           0.96       220
   macro avg       0.98      0.79      0.86       220
weighted avg       0.97      0.96      0.96       220

 97.355015 seconds (53.70 M allocations: 52.806 GiB, 9.27% gc time)
Accuracy score: 0.9555302166476625

3260-01D done
 ******************************* 
 ******************************* 
Nombre de modèles à parcourir: 912384

Fitting 5 folds for each of 250 ca

In [ ]:
"""
getLogitPCA permet d'obtenir pour chaque ouvrage un modèle de prédiction basé sur la régression logistique et l'analyse
en composantes principales. La fonction effectue une grid search sur chacune des configurations possibles proposées
par le dictionnaire des hyoerparamètres fit_params

@params
num_ouvrage:      le numéro de l'ouvrage à évaluer
fit_param:        dictionnaire des valeurs possibles pour les hyperparamètres du modèle de régression logistique
n_folds:          le nombre de plis à effectuer sur l'ensemble d'entrainement lors de la validation croisée

"""

function getLogitPCA(num_ouvrage::String; fit_params::Dict, n_folds=5, verbose=1)
    train = CreateDataframe(num_ouvrage, surverse_df)
    all_X = convert(Matrix{Float64}, train[:, 4:28])
    all_y = train[!, :SURVERSE]
    
    # On doit utiliser la fonction train_test_split de sklearn car notre jeu de données est très désiquilibré.
    # le parametre "stratify" de la fonction permet de parer à ca
    
    X_train, X_test, y_train, y_test = train_test_split(all_X, all_y, test_size=.2, shuffle=true, random_state = 34, stratify=all_y)

    n_models = 1
    
    for (k,v) in fit_params
        n_models *= length(v)
    end

    println("Nombre de modèles à parcourir: $(n_models)\n")
    
    # Préparation du Pipeline à tester.
    # Le Pipeline permet de lier la standardisation des données à l'obtention des composantes principales et à la création 
    # d'une régression logistique les utilisant.
    pipe = Pipelines.Pipeline([("ss", StandardScaler()),
                               ("pca", PCA()),
                               ("lr", LogisticRegression(max_iter=1000, random_state=42))])
    
    M = GridSearch.GridSearchCV(estimator=pipe,
                                param_grid=fit_params,
                                cv=n_folds, 
                                verbose=verbose)

    return fit!(M, X_train, y_train)
end


In [ ]:
lr_fit_params = Dict()
lr_fit_params["lr__C"] = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]#exp10.(-4: 1: 1)
lr_fit_params["pca__n_components"] = 2:1:15

ouvrage = ["3260-01D", "3350-07D", "4240-01D", "4350-01D", "4380-01D"]
best_models_lr = Dict()

for i=1:length(ouvrage)
    @time best_models_lr[ouvrage[i]] = getLogitPCA(ouvrage[i], fit_params=lr_fit_params)
    println("Accuracy score: ", best_models_lr[ouvrage[i]].best_score_)
    println(ouvrage[i], " done\n")
end


In [ ]:
# PAS SÛR À 100% QU'IL FAILLE VRAIMENT FAIRE ÇA MAIS ÇA ME SEMBLAIT PAS MAL
# On choisit maintenant, pour chacun des ouvrages, le modèle ayant la meilleur accuracy entre la RandomForest et la 
# régression logistique

# best_models = Dict()

# for (ouvrage, modele) in best_models_rf
#     if modele.best_score_ > best_models_lr[ouvrage].best_score_
#         best_models[ouvrage] = modele
#     else
#         best_models[ouvrage] = best_models_lr[ouvrage]
#     end
# end
best_models = best_models_rf

# Création du fichier de prédictions pour soumettre sur Kaggle

Dans ce cas-ci, nous prédirons une surverse avec une probabilité de 1/2 sans considérer aucune variable explicative.

In [ ]:
function getVarEx(date::Date)
    row_sum = filter(row-> row.date== date, X_pcp_sum)
    row_sum_2_days = filter(row-> row.date== date, X_pcp_two_days)
    row_max = filter(row-> row.date== date, X_pcp_max)
    row_max_3h = filter(row->row.date== date, X_max_3h)
    row_bloc_3h = filter(row->row.date==date, X_bloc_3h)
    total_row = join(row_sum, row_sum_2_days, on = :date)
    total_row = join(total_row, row_max, on = :date)
    total_row = join(total_row, row_max_3h, on = :date)
    total_row = join(total_row, row_bloc_3h, on = :date)
    
    return total_row
end

In [ ]:
## Pas classe, je sais
function dfRowTo2D(row::DataFrameRow)
    X = Array{Float64, 2}(undef, 1, length(row))
    for i=1:length(row)
        X[1, i] = row[i]
    end
    return X
end

In [ ]:
## Chargement du fichier de test
test = CSV.read("data/test.csv")
hcat(test, DataFrame(SURVERSE=zeros(Int64, length(test[:,1]))))
rename!(test, :DATE => :date)

## Dataframe des dates et ouvrages à prédire avec toutes les variables explicatives
df_test = getVarEx(test[1, :date])
for i=2:length(test[:,1])
    push!(df_test, convert(Array, getVarEx(test[i, :date])))
end
insert!(df_test, 1, test[:, :NO_OUVRAGE], :NO_OUVRAGE)
insert!(df_test, 3, zeros(Int64, length(df_test[:,1])), :surverse)
#insert!(df_test, 4, ones(Float64, length(df_test[:,1])), :x1)
ambig = []
## Calcul de la probabilité de surverse
for i=1:length(df_test[:,1])
    M = best_models[df_test[i, :NO_OUVRAGE]]
    Exp = dfRowTo2D(df_test[i,4:28])
    proba = predict_proba(M, Exp)
    prediction = predict(M, Exp)
    ouvr = "$(df_test[i, :NO_OUVRAGE])  on  $(df_test[i, :date])"
    if all(i -> i < 0.65, proba) #On print les jours ambiguës
        #push!(ambig, (ouvr, proba))
        println(ouvr)
        println(proba)
        println(prediction, "\n")
    end
    
    df_test[i, :surverse] = prediction[1]
end
#println(ambig)
# Création du fichier sampleSubmission.csv pour soumettre sur Kaggle
ID = test[:,:NO_OUVRAGE].*"_".*string.(test[:,:date])
sampleSubmission = DataFrame(ID = ID, Surverse=df_test[:,:surverse])
CSV.write("sampleSubmission_with_5_variables_rfpca_rand.csv",sampleSubmission)

# Vous pouvez par la suite déposer le fichier sampleSubmission.csv sur Kaggle.